# Check if all required packages work

In [1]:
import numpy as np
import cvxpy as cp
from cc_utils import compute_gram_matrix, cholesky_decomposition, mmd_eps

# Convex approximation of Chance Constraints using the CVaR

In [2]:
# Set problem parameters
dim_x = 3

# Define problem related constants and variables
# risk level
alpha = 0.05

# Cost for our portfolio
c = np.arange(1, 1 + dim_x * 0.5, 0.5)
def objective(x):
    return c@x

In [3]:
# Define uncertain constraint function for our problem
def cvx_f_constraint(x, X):
    """
    Constraint function

    x: cp.Variable -- decision variable (dim,)
    X: ndarray -- Samples (n_samples, dim)
    """
    f = cp.square(X @ x) - 1
    return f

In [4]:
# Define additional deterministic constraints
def x_constraints(x):
    return [
        x >= 0,
        cp.sum(x) <= 1
    ]

In [5]:
# Create samples of the uncertainty from a multivariate gaussian
mean = np.zeros(dim_x)
cov = np.zeros((dim_x, dim_x))
np.fill_diagonal(cov, np.arange(0.5, 0.5*(dim_x + 1), 0.5))

n_samples = 500

samples = np.random.multivariate_normal(mean, cov, size=n_samples)
test_samples = np.random.multivariate_normal(mean, cov, size=100000)

# Scenario Optimization

In [6]:
x = cp.Variable(shape=(dim_x, 1), name='cash')

In [7]:
# Evaluate constraints for our problem
constraints = x_constraints(x)  # returns a list
f_const = cvx_f_constraint(x, samples)
constraints.append(f_const <= 0)

In [8]:
# Define cvx problem object
prob = cp.Problem(objective=cp.Maximize(objective(x)),
                  constraints=constraints)

try:
    prob.solve()
except:
    raise ValueError("Optimization failed.")

In [9]:
obj_scen_sol = prob.value
x_scen_sol = x.value

# CVaR approximation of CC program

In [10]:
# decision variables
x = cp.Variable(shape=(dim_x, 1), name='cash')
t = cp.Variable(1, name='CVaR')

In [11]:
# Setup constraint for our defined decision variable
constraints = x_constraints(x)
f_const = cvx_f_constraint(x, samples)

# Add the CVaR constraint accoring to its dual defintion
# Use cp.sum and cp.maximum to compute the CVaR
# We do not need to evalute inf_t here but only need to find one variabel t such that constraint is satisfied
constraints.append(cp.sum(cp.maximum(f_const + t, 0))/n_samples <= t*alpha)


# Set up cvx object and solve it
prob = cp.Problem(objective=cp.Maximize(objective(x)),
                  constraints=constraints)
try:
    prob.solve()
except:
    raise ValueError("Optimization failed.")

In [12]:
# Retrieve solution
obj_cvar_sol = prob.value
x_cvar_sol = x.value

# Distributionally Robust Chance Constraint

In [13]:
# compute empirical CVaR using a Monte-Carlo approximation of expectation
t = cp.Variable(1, 'CVaR')
f_cvar = cvx_f_constraint(x, test_samples).value
# Define CVaR as objective now
cvar_obj = cp.sum(cp.maximum(f_cvar + t, 0)) / test_samples.shape[0] - t * alpha
prob = cp.Problem(objective=cp.Minimize(cvar_obj))
try:
    prob.solve()
    emp_cvar = prob.value
except:
    raise ValueError("Could not compute CVaR.")

In [14]:
# Set up DR-CVaR problem using MMD ambiguity sets
kernel_param = {'kernel': 'rbf'}

# Compute the ambiguity set radius
epsilon = mmd_eps(n_sample=n_samples,
                  alpha=alpha)

# Precompute kernel matrix
kernel_matrix = compute_gram_matrix(samples, param=kernel_param)
kernel_cholesky = cholesky_decomposition(kernel_matrix)

# Define variables
x = cp.Variable(shape=(dim_x, 1), name='cash')
t = cp.Variable(1, name='CVaR')
w = cp.Variable(shape=(n_samples, 1), name='weights')
g0 = cp.Variable(1, name='g0')

# Define constraints
g_rkhs = kernel_matrix @ w
Eg_rkhs = 1/n_samples * cp.sum(g_rkhs[:n_samples])
g_norm = cp.norm(kernel_cholesky @ w)
f_const = cvx_f_constraint(x, samples)

constraints = x_constraints(x)
constraints.extend([
    g0 + Eg_rkhs + epsilon * g_norm <= t * alpha,
    cp.pos(f_const + t) <= g0 + g_rkhs
])

In [15]:
# Set up problem object and solve it
prob = cp.Problem(objective=cp.Maximize(objective(x)),
                  constraints=constraints)
try:
    prob.solve()
except:
    raise ValueError("Optimization failed.")
obj_sol = prob.value
x_sol = x.value

In [16]:
# compute empirical CVaR using the test samples
t = cp.Variable(1, 'CVaR')
f_cvar = cvx_f_constraint(x, test_samples).value
cvar_obj = cp.sum(cp.maximum(f_cvar + t, 0)) / test_samples.shape[0] - t * alpha
prob = cp.Problem(objective=cp.Minimize(cvar_obj))
try:
    prob.solve()
    emp_drcvar = prob.value
except:
    raise ValueError("Could not compute CVaR.")

print(emp_drcvar)

-0.019123601356174344


In [17]:
print("Scenario solution: \t{0} \t x -- {1}".format(obj_scen_sol, x_scen_sol.T))
print("CVaR approximation: \t{0} \t x -- {1} \t CVaR -- {2}".format(obj_cvar_sol, x_cvar_sol.T, np.round(emp_cvar, 5)))
print("DR-CVaR approximation: \t{0} \t x -- {1} \t CVaR -- {2}".format(obj_sol, x_sol.T, np.round(emp_drcvar, 5)))

Scenario solution: 	0.7299754362872397 	 x -- [[0.18422131 0.29218825 0.05373588]]
CVaR approximation: 	1.1208077520550666 	 x -- [[0.42092581 0.23287026 0.17528828]] 	 CVaR -- 0.00294
DR-CVaR approximation: 	0.8494535530150683 	 x -- [[0.26640989 0.23111511 0.1181855 ]] 	 CVaR -- -0.01912
